# Creating A Spotify Database from Scratch

 Let's begin by importing needed packages and reading in the Spotify data.

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from IPython.display import HTML
from sqlalchemy import create_engine

In [2]:
# Read in spotify data into a dataframe
df = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')
df.columns
pd.set_option('display.max_columns', 30)
subset = df.sample(5)

In [3]:
HTML(subset.to_html())

In [4]:
df[['track_id', 'track_album_id', 'track_album_name']]

track_id          track_album_id  \
0      6f807x0ima9a1j3VPbc7VN  2oCs0DGTsRO98Gh5ZSl2Cx   
1      0r7CVbZTWZgbTCYdfa2P31  63rPSO264uRjW1X5E6cWv6   
2      1z1Hg7Vb0AhHDiEmnDE79l  1HoSmj2eLcsrR0vE9gThr4   
3      75FpbthrwQmzHlBJLuGdC7  1nqYsOef1yKKuGOVchbsk6   
4      1e8PAfcKUYoKkxPhrHqw4x  7m7vv9wlQ4i0LFuJiE2zsQ   
...                       ...                     ...   
32828  7bxnKAamR3snQ1VGLuVfC1  2azRoBBWEEEYhqV6sb7JrT   
32829  5Aevni09Em4575077nkWHz  6kD6KLxj7s8eCE3ABvAyf5   
32830  7ImMqPP3Q1yfUHvsdn7wEo  0ltWNSY9JgxoIZO4VzuCa6   
32831  2m69mhnfQ1Oq6lGtXuYhgX  1fGrOkHnHJcStl14zNx8Jy   
32832  29zWqhca3zt5NsckZqDf6c  0X3mUOm6MhxR7PzxG95rAo   

                                        track_album_name  
0      I Don't Care (with Justin Bieber) [Loud Luxury...  
1                        Memories (Dillon Francis Remix)  
2                        All the Time (Don Diablo Remix)  
3                            Call You Mine - The Remixes  
4                Someone You Loved (Future Humans Remix)  
...                                                  ...  
32828                         City Of Lights (Vocal Mix)  
32829                                     Closer Remixed  
32830                       Sweet Surrender (Radio Edit)  
32831                             Only For You (Remixes)  
32832                                      Typhoon/Storm  

[32833 rows x 3 columns]

The first step in getting this data into a database is normalizing it. Normalization makes the database more efficient and flexible by reducing the amount of redundant data. The database is already in first normal form, as there are no columns that contain multiple data elements; the next step is to determine how to get it into second normal form. Second normal form is when all of the non-key columns are dependent only on the table's primary key. This sounds a bit jargony, I know, but an example may help clarify things. 

A primary key is a value that is unique for each record in the database. In the current scenario, we have a unique track id for every track in our dataset, so we could potentially use this as our primary key. If the database was in second normal form all the other columns in the table would only be dependent on the track id. This is not the case currently, as the present columns are dependent on other values in the database. For example, the track album name is dependent on the track album id. We need to break the table up into smaller tables to fix this problem.

| Table Name  | Contents    |
| ----------- | ----------- |
| tracks       | track id and song characteristics|
| album_name  | album name and album id       |
| album release | album id and release date|
| playlist  | playlist id and playlist characteristics|
| track_playlist | track id and playlist id |
| track_artist | track id and artist name|

In [5]:
# Create subset dataframes to be turned into individual tables
tracks = df[['track_id', 'track_popularity', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_ms']]

track_name = df [['track_id', 'track_name']]

track_album = df[['track_album_id', 'track_id']]

album_name = df[['track_album_id', 'track_album_name']]

album_release = df[['track_album_release_date', 'track_album_id']]

playlist_name = df[['playlist_name', 'playlist_id']]

playlist_genre = df[['playlist_id', 'playlist_genre']]

playlist_subgenre = df[['playlist_id', 'playlist_subgenre']]

track_playlist = df[['playlist_id', 'track_id']]

track_artist = df[['track_id', 'track_artist']]

In [6]:
# Create a database to connect to in memory
cnx = sqlite3.connect('spotify.db')

# Create the tables in the database
tracks.to_sql(name='tracks', con=cnx, index = False)
track_name.to_sql(name='track_name', con=cnx, index = False)
playlist_name.to_sql(name='playlist_name', con = cnx, index = False)
playlist_genre.to_sql(name='playlist_genre', con = cnx, index=False)
playlist_subgenre.to_sql(name = 'playlist_subgenre', con = cnx, index=False)
album_release.to_sql(name='album_release', con = cnx, index = False)
album_name.to_sql(name='album_name', con = cnx, index = False)
track_album.to_sql(name='track_album', con = cnx, index = False)
track_playlist.to_sql(name = 'track_playlist', con = cnx, index = False)
track_artist.to_sql(name= 'track_artist', con = cnx, index = False)

 Load SQL and connect to our new Spotify database.

In [7]:
%load_ext sql

In [8]:
%sql sqlite:///spotify.db

'Connected: @spotify.db'

Examine the structure of the SQL database tables.

In [9]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///spotify.db
Done.


[('table', 'tracks', 'tracks', 2, 'CREATE TABLE "tracks" (\n"track_id" TEXT,\n  "track_popularity" INTEGER,\n  "danceability" REAL,\n  "energy" REAL,\n  "key" INTEGER,\n  "loudness" RE ... (32 characters truncated) ... hiness" REAL,\n  "acousticness" REAL,\n  "instrumentalness" REAL,\n  "liveness" REAL,\n  "valence" REAL,\n  "tempo" REAL,\n  "duration_ms" INTEGER\n)'),
 ('table', 'track_name', 'track_name', 952, 'CREATE TABLE "track_name" (\n"track_id" TEXT,\n  "track_name" TEXT\n)'),
 ('table', 'playlist_name', 'playlist_name', 1340, 'CREATE TABLE "playlist_name" (\n"playlist_name" TEXT,\n  "playlist_id" TEXT\n)'),
 ('table', 'playlist_genre', 'playlist_genre', 1797, 'CREATE TABLE "playlist_genre" (\n"playlist_id" TEXT,\n  "playlist_genre" TEXT\n)'),
 ('table', 'playlist_subgenre', 'playlist_subgenre', 2072, 'CREATE TABLE "playlist_subgenre" (\n"playlist_id" TEXT,\n  "playlist_subgenre" TEXT\n)'),
 ('table', 'album_release', 'album_release', 2413, 'CREATE TABLE "album_release" (\n"track_album_release_date" TEXT,\n  "track_album_id" TEXT\n)'),
 ('table', 'album_name', 'album_name', 2739, 'CREATE TABLE "album_name" (\n"track_album_id" TEXT,\n  "track_album_name" TEXT\n)'),
 ('table', 'track_album', 'track_album', 3129, 'CREATE TABLE "track_album" (\n"track_album_id" TEXT,\n  "track_id" TEXT\n)'),
 ('table', 'track_playlist', 'track_playlist', 3554, 'CREATE TABLE "track_playlist" (\n"playlist_id" TEXT,\n  "track_id" TEXT\n)'),
 ('table', 'track_artist', 'track_artist', 3979, 'CREATE TABLE "track_artist" (\n"track_id" TEXT,\n  "track_artist" TEXT\n)')]

In [10]:
%%sql
SELECT *
FROM tracks
LIMIT 5

 * sqlite:///spotify.db
Done.


[('6f807x0ima9a1j3VPbc7VN', 66, 0.748, 0.9159999999999999, 6, -2.634, 1, 0.0583, 0.102, 0.0, 0.0653, 0.518, 122.036, 194754),
 ('0r7CVbZTWZgbTCYdfa2P31', 67, 0.726, 0.815, 11, -4.968999999999999, 1, 0.0373, 0.0724, 0.00421, 0.35700000000000004, 0.693, 99.97200000000001, 162600),
 ('1z1Hg7Vb0AhHDiEmnDE79l', 70, 0.675, 0.9309999999999999, 1, -3.432, 0, 0.0742, 0.0794, 2.33e-05, 0.11, 0.613, 124.008, 176616),
 ('75FpbthrwQmzHlBJLuGdC7', 60, 0.718, 0.93, 7, -3.778, 1, 0.102, 0.0287, 9.43e-06, 0.204, 0.27699999999999997, 121.956, 169093),
 ('1e8PAfcKUYoKkxPhrHqw4x', 69, 0.65, 0.833, 1, -4.672, 1, 0.0359, 0.0803, 0.0, 0.0833, 0.725, 123.976, 189052)]

In [11]:
%%sql
SELECT * 
FROM track_name
LIMIT 5

 * sqlite:///spotify.db
Done.


[('6f807x0ima9a1j3VPbc7VN', "I Don't Care (with Justin Bieber) - Loud Luxury Remix"),
 ('0r7CVbZTWZgbTCYdfa2P31', 'Memories - Dillon Francis Remix'),
 ('1z1Hg7Vb0AhHDiEmnDE79l', 'All the Time - Don Diablo Remix'),
 ('75FpbthrwQmzHlBJLuGdC7', 'Call You Mine - Keanu Silva Remix'),
 ('1e8PAfcKUYoKkxPhrHqw4x', 'Someone You Loved - Future Humans Remix')]

In [12]:
%%sql
SELECT *
FROM playlist
LIMIT 5

 * sqlite:///spotify.db
(sqlite3.OperationalError) no such table: playlist
[SQL: SELECT *
FROM playlist
LIMIT 5]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


In [13]:
%%sql
SELECT *
FROM album_release
LIMIT 5

 * sqlite:///spotify.db
Done.


[('2019-06-14', '2oCs0DGTsRO98Gh5ZSl2Cx'),
 ('2019-12-13', '63rPSO264uRjW1X5E6cWv6'),
 ('2019-07-05', '1HoSmj2eLcsrR0vE9gThr4'),
 ('2019-07-19', '1nqYsOef1yKKuGOVchbsk6'),
 ('2019-03-05', '7m7vv9wlQ4i0LFuJiE2zsQ')]

In [14]:
%%sql
SELECT * 
FROM album_name
LIMIT 5

 * sqlite:///spotify.db
Done.


[('2oCs0DGTsRO98Gh5ZSl2Cx', "I Don't Care (with Justin Bieber) [Loud Luxury Remix]"),
 ('63rPSO264uRjW1X5E6cWv6', 'Memories (Dillon Francis Remix)'),
 ('1HoSmj2eLcsrR0vE9gThr4', 'All the Time (Don Diablo Remix)'),
 ('1nqYsOef1yKKuGOVchbsk6', 'Call You Mine - The Remixes'),
 ('7m7vv9wlQ4i0LFuJiE2zsQ', 'Someone You Loved (Future Humans Remix)')]

In [15]:
%%sql
SELECT * 
FROM track_album
LIMIT 5

 * sqlite:///spotify.db
Done.


[('2oCs0DGTsRO98Gh5ZSl2Cx', '6f807x0ima9a1j3VPbc7VN'),
 ('63rPSO264uRjW1X5E6cWv6', '0r7CVbZTWZgbTCYdfa2P31'),
 ('1HoSmj2eLcsrR0vE9gThr4', '1z1Hg7Vb0AhHDiEmnDE79l'),
 ('1nqYsOef1yKKuGOVchbsk6', '75FpbthrwQmzHlBJLuGdC7'),
 ('7m7vv9wlQ4i0LFuJiE2zsQ', '1e8PAfcKUYoKkxPhrHqw4x')]

Let's try a sample query! I'm interested in the average danceability of songs by playlist genre. What playlist is the most danceable?

In [24]:
%%sql
SELECT AVG(tracks.danceability) as Danceability, playlist_genre.playlist_genre as Genre
FROM tracks
JOIN playlist_genre ON playlist_genre.playlist_id = track_playlist.playlist_id
JOIN track_playlist ON track_playlist.track_id = tracks.track_id
GROUP BY playlist_genre.playlist_genre
ORDER by AVG(tracks.danceability) DESC

 * sqlite:///spotify.db
Done.


[(0.7170314824278323, 'rap'),
 (0.7079193688810412, 'latin'),
 (0.6716703849596874, 'r&b'),
 (0.6547498244396701, 'edm'),
 (0.6406080105401385, 'pop'),
 (0.5259998712902841, 'rock')]

In [17]:
%%sql
SELECT DISTINCT(track_name.track_name) as TrackName, (tracks.danceability) as Danceability, playlist_genre.playlist_genre as Genre
FROM tracks
JOIN playlist_genre ON playlist_genre.playlist_id = track_playlist.playlist_id
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_playlist ON track_playlist.track_id = tracks.track_id
WHERE playlist_genre.playlist_genre = 'rap'
ORDER by (tracks.danceability) DESC
LIMIT 5

 * sqlite:///spotify.db
Done.


[('Funky Friday', 0.975, 'rap'),
 ('Bad Bad Bad (feat. Lil Baby)', 0.9740000000000001, 'rap'),
 ('In da Wind', 0.9740000000000001, 'rap'),
 ('Skiety & Klapki - Remix', 0.972, 'rap'),
 ('Sake', 0.971, 'rap')]

It seems a little strange that rap is the most danceable music... I wonder if something else is driving this?

For fun, let's look at the least danceable music! It turns out it's a track called, "Hi, How're You Doin'?" by DREAMS COME TRUE. It's just a 10-second recording of someone saying "Hi, How're You Doin?' The rating seems about right. 

In [18]:
%%sql
SELECT DISTINCT(track_name.track_name), track_artist.track_artist, tracks.energy
FROM tracks
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_artist ON track_artist.track_id = tracks.track_id
ORDER BY tracks.energy DESC
LIMIT 10

 * sqlite:///spotify.db
Done.


[('Rain Forest and Tropical Beach Sound', 'Nature Sounds Nature Music', 1.0),
 ('Chill Waves & Wind in Leaves', 'Pinetree Way', 1.0),
 ('Forest Rain', 'Rain Recordings', 1.0),
 ('Captain Jack - Short Mix', 'Captain Jack', 0.9990000000000001),
 ('Staatsfeind', 'Agonoize', 0.9990000000000001),
 ('Immortal - Single Edit', 'Solitary Experiments', 0.9990000000000001),
 ('Run To You', 'Code 64', 0.9990000000000001),
 ('Satisfaction - RL Grime Remix', 'Benny Benassi', 0.9990000000000001),
 ('Nightfall by the Sea', 'Shaman', 0.9990000000000001),
 ('Gentle Waves on Rocks', 'Pinetree Way', 0.9990000000000001)]

The most popular tracks of our time.

In [19]:
%%sql
SELECT DISTINCT(track_name.track_name), tracks.track_popularity, track_artist.track_artist
FROM tracks
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_artist ON track_artist.track_id = tracks.track_id
ORDER BY tracks.track_popularity DESC
LIMIT 10

 * sqlite:///spotify.db
Done.


[('Dance Monkey', 100, 'Tones and I'),
 ('ROXANNE', 99, 'Arizona Zervas'),
 ('Tusa', 98, 'KAROL G'),
 ('Memories', 98, 'Maroon 5'),
 ('Blinding Lights', 98, 'The Weeknd'),
 ('Circles', 98, 'Post Malone'),
 ('The Box', 98, 'Roddy Ricch'),
 ('everything i wanted', 97, 'Billie Eilish'),
 ("Don't Start Now", 97, 'Dua Lipa'),
 ('Falling', 97, 'Trevor Daniel')]

In [20]:
%%sql
SELECT AVG(tracks.valence), playlist_genre.playlist_genre
FROM tracks
JOIN track_name ON track_name.track_id = tracks.track_id
JOIN track_artist ON track_artist.track_id = tracks.track_id
JOIN playlist_genre ON playlist_genre.playlist_id = track_playlist.playlist_id
JOIN track_playlist ON track_playlist.track_id = tracks.track_id
GROUP BY playlist_genre.playlist_genre
ORDER BY AVG(tracks.valence) DESC

 * sqlite:///spotify.db
Done.


[(0.5672185249397307, 'rock'),
 (0.524778251918615, 'latin'),
 (0.5072517434237556, 'pop'),
 (0.4933969699857038, 'r&b'),
 (0.4612765718159442, 'rap'),
 (0.45100491301185863, 'edm')]

Use an SQL query to find the names of all playlists that contain instrumentals.

In [21]:
%%sql 
SELECT PlaylistName, NumberInstrumentals
FROM (SELECT COUNT(DISTINCT(tracks.track_id)) as NumberInstrumentals, playlist_name.playlist_name as PlaylistName
      FROM tracks
      JOIN track_playlist ON track_playlist.track_id = tracks.track_id
      JOIN track_name ON track_name.track_id = tracks.track_id
      JOIN track_artist ON track_artist.track_id = tracks.track_id
      JOIN playlist_name ON playlist_name.playlist_id = track_playlist.playlist_id
      WHERE tracks.instrumentalness >.50
      GROUP BY playlist_name.playlist_name
     )
WHERE NumberInstrumentals > 0
ORDER BY NumberInstrumentals DESC
LIMIT 10

 * sqlite:///spotify.db
Done.


[('Lush Lofi', 91),
 ('Underground Party | Hypnotic | Minimal | Acid | Big Room | Tech | Liquid', 88),
 ('House Electro 2019', 81),
 ('Jazz Vibes', 81),
 ('Lo-Fi Beats', 78),
 ('Lofi Hip-Hop', 75),
 ('Selected House', 60),
 ('💊ELECTRO-HOUSE-TECH💊', 58),
 ('Sunny Beats', 54),
 ('Tropical Vibes', 52)]

Find the total number of playlists containing instrumentals. 

In [22]:
%%sql 
SELECT COUNT(DISTINCT(PlaylistName))
FROM (SELECT COUNT(DISTINCT(tracks.track_id)) as NumberInstrumentals, playlist_name.playlist_name as PlaylistName
      FROM tracks
      JOIN track_playlist ON track_playlist.track_id = tracks.track_id
      JOIN track_name ON track_name.track_id = tracks.track_id
      JOIN track_artist ON track_artist.track_id = tracks.track_id
      JOIN playlist_name ON playlist_name.playlist_id = track_playlist.playlist_id
      WHERE tracks.instrumentalness >.50
      GROUP BY playlist_name.playlist_name
     )
WHERE NumberInstrumentals > 0
LIMIT 10

 * sqlite:///spotify.db
Done.


[(256,)]